In [1]:
import tkinter as tk
from tkinter import filedialog, scrolledtext
import os
import ollama

def load_file():
    # Deschide dialogul pentru selectarea fișierului
    filepath = filedialog.askopenfilename(title="Selectează un fișier")
    if not filepath:
        return  # utilizatorul a anulat selecția

    try:
        # Citește fișierul în modul binar
        with open(filepath, "rb") as f:
            file_bytes = f.read()
        
        # Informații macro: nume și dimensiune
        file_size = os.path.getsize(filepath)
        file_name = os.path.basename(filepath)
        macro_info = f"Nume fișier: {file_name}\nDimensiune: {file_size} bytes\n"
        
        # Informații micro: hex dump pentru primele 256 de octeți
        hex_dump = file_bytes[:256].hex()
        micro_info = f"Hex dump (primele 256 de octeți):\n{hex_dump}\n"
        
        # Combină informațiile pentru analiza modelului
        analysis_info = macro_info + "\n" + micro_info
        
        # Afișează informațiile despre fișier în text_area
        text_area.delete(1.0, tk.END)
        text_area.insert(tk.END, "Fișier încărcat cu succes:\n")
        text_area.insert(tk.END, analysis_info)
        
        # Pregătește prompt-ul pentru model
        prompt = (
            "Ești un expert în analiza malware. Analizează următoarele detalii "
            "ale unui fișier și determină dacă este suspect sau chiar malware. "
            "Oferă un rezumat la nivel macro și micro și justifică evaluarea ta.\n\n"
            f"{analysis_info}"
        )
        
        # Apelează modelul local "mistral" prin Ollama
        response = ollama.chat(
            model='mistral',
            messages=[{'role': 'user', 'content': prompt}],
            stream=False,
        )
        
        result = response.get('message', {}).get('content', 'Nu s-a putut obține un răspuns de la model.')
        text_area.insert(tk.END, "\n\nAnaliza modelului:\n")
        text_area.insert(tk.END, result)
        
    except Exception as e:
        text_area.insert(tk.END, f"\n\nEroare: {e}")

# Configurarea interfeței Tkinter
root = tk.Tk()
root.title("Analiza fișier Malware")

# Butonul pentru încărcarea fișierului
load_btn = tk.Button(root, text="Încarcă fișier", command=load_file)
load_btn.pack(pady=10)

# Zonă text cu scroll pentru afișarea informațiilor și a rezultatului analizei
text_area = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=80, height=30)
text_area.pack(padx=10, pady=10)

root.mainloop()
